# Data Cleaning
---
Take the raw data from `/data/hamSpam.csv` and `/data/phish.csv` and conform the ham/spam dataset to the structure of the phishing dataset and integrate together. Then clean the dataset by removing NA values, standardize the textual features by lowercasing and removing stopwords, and remove any embedded HTML elements from the text.

## 1. Imports & Setup

In [1]:
import pandas as pd

import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

from utils import clean_text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/prokope/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Download nltk datasets.

In [2]:
# NLTK downloads
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/prokope/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/prokope/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/prokope/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## 2. Load Data from CSV

In [14]:
hamSpam_df = pd.read_csv("./data/1_hamSpam.csv")
phish_df = pd.read_csv("./data/1_phish.csv")

## 3. Process Ham/Spam Data

In [29]:
# Rename known columns (no error if source names are missing)
hamSpam_df.rename(
    columns={"label": "Email Type", "text": "Email Text"},
    inplace=True
)

hamSpam_df["Email Type"] = hamSpam_df["Email Type"].str.lower()

# Export as processed dataset
hamSpam_df.to_feather(
    "data/2_hamSpam_processed.feather"
)

## 4. Process Phishing Data

Observe email types.

In [15]:
phish_df["label"].apply(repr).unique()

array(['1'], dtype=object)

Standardize email types.

In [27]:
other_columns = ["subject", "sender", "receiver", "date", "urls"]
if set(other_columns).intersection(phish_df.columns) == other_columns:
    phish_df.drop(columns=["subject", "sender", "receiver", "date", "urls"], inplace=True)

phish_df.rename(
    columns={"label": "Email Type", "body": "Email Text"},
    inplace=True
)

phish_df['Email Type'] = phish_df['Email Type'].replace(
    {1: 'phish'}
)

phish_df['Email Type'].apply(repr).unique()

array(["'phish'"], dtype=object)

## 5. Combine Datasets

In [30]:
email_df = pd.concat([hamSpam_df, phish_df], ignore_index=True)
email_df['Email Type'].apply(repr).unique()

array(["'spam'", "'ham'", "'phish'"], dtype=object)

Display examples of each category.

In [31]:
email_df[email_df["Email Type"] == "spam"].iloc[[5000]]

,Email Type,Email Text
10593,spam,"from : mr frank fido\ndirector , delivery / op..."


In [32]:
email_df[email_df["Email Type"] == "ham"].iloc[[5000]]

,Email Type,Email Text
9463,ham,could please together confidentiality agreemen...


In [33]:
email_df[email_df["Email Type"] == "phish"].iloc[[5000]]

,Email Type,Email Text
198852,phish,"GOODDAY,\n\nWE ARE VERY SORRY IF THIS MIGHT OF..."


Show total count of emails.

In [34]:
email_df.count()

Email Type    221049
Email Text    221047
dtype: int64

Show counts for each category.

In [35]:
email_df["Email Type"].value_counts()

Email Type
ham      102160
spam      91692
phish     27197
Name: count, dtype: int64

## 6. Drop NA and "empty" Rows

In [37]:
print(email_df.isna().sum())

email_df.dropna(inplace=True)

empty_rows = email_df[email_df["Email Text"] == "empty"]
email_df.drop(empty_rows.index, inplace=True)

print(email_df.isna().sum())

Email Type    0
Email Text    2
dtype: int64
Email Type    0
Email Text    0
dtype: int64


## 7. Drop Any Duplicate Rows
`spam` contains many duplicated rows, which will cause the model to overfit to spam email data.

In [40]:
email_df[email_df["Email Type"] == "spam"][email_df.duplicated() == True].count()

/tmp/ipykernel_24409/2249439977.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  email_df[email_df["Email Type"] == "spam"][email_df.duplicated() == True].count()


Email Type    0
Email Text    0
dtype: int64

In [41]:
email_df[email_df["Email Type"] == "ham"][email_df.duplicated() == True].count()

/tmp/ipykernel_24409/1286218904.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  email_df[email_df["Email Type"] == "ham"][email_df.duplicated() == True].count()


Email Type    0
Email Text    0
dtype: int64

In [42]:
email_df[email_df["Email Type"] == "phish"][email_df.duplicated() == True].count()

/tmp/ipykernel_24409/136315833.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  email_df[email_df["Email Type"] == "phish"][email_df.duplicated() == True].count()


Email Type    1
Email Text    1
dtype: int64

Drop any duplicates and confirm that count of duplicates is 0.

In [43]:
email_df.drop_duplicates(keep="first", inplace=True)
email_df[email_df.duplicated() == True].count()

Email Type    0
Email Text    0
dtype: int64

Get total count of dataset after removing duplicates

In [44]:
email_df["Email Type"].value_counts()

Email Type
ham      102159
spam      91691
phish     27196
Name: count, dtype: int64

## 7. Bulk Clean Dataset & Export to CSV


Clean data by removing stop words, punctuation, special characters, and tokenize/lemmatize.

In [45]:
cleaned_email_df = email_df.copy()
cleaned_email_df["Cleaned Email Text"] = email_df["Email Text"].apply(clean_text)
cleaned_email_df.head()

,Email Type,Email Text,Cleaned Email Text
0,spam,viiiiiiagraaaa\nonly for the ones that want to...,viiiiiiagraaaa one want make scream prodigy sc...
1,ham,got ice thought look az original message ice o...,got ice thought look az original message ice o...
2,spam,yo ur wom an ne eds an escapenumber in ch ma n...,yo ur wom ne ed escapenumber ch n b e th n f h...
3,spam,start increasing your odds of success & live s...,start increasing odds success live sexually he...
4,ham,author jra date escapenumber escapenumber esca...,author jra date escapenumber escapenumber esca...


Export to feather.

In [46]:
cleaned_email_df.to_feather(
    "./data/2_clean_email_dataset.feather"
)